In [1]:
!pip install mpi4py

     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074471 sha256=45e30ca6d11682481a21091041120f6e380138474b68d6c341e544788cf5005f
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py


How does a process know where to send a message?

In [2]:
%%writefile mpi1.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print('size=%d, rank=%d' % (size, rank))

Writing mpi1.py


In [3]:
!mpiexec --allow-run-as-root -n 4 python mpi1.py

size=4, rank=2
size=4, rank=0
size=4, rank=1
size=4, rank=3



One MPI program, multiple MPI processes

In [4]:
%%writefile mpi2.py

from mpi4py import MPI
rank = MPI.COMM_WORLD.Get_rank()

a = 6.0
b = 3.0
if rank == 0:
        print(a + b)
if rank == 1:
        print(a * b)
if rank == 2:
        print(max(a,b))
print("end MPI")

Writing mpi2.py


In [15]:
!mpiexec --allow-run-as-root -n 3 python mpi2.py

18.0
end MPI
6.0
end MPI
9.0
end MPI


Point-to-point communication

In [6]:
%%writefile mpi3.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("Process", rank, "received the number", randNum[0])


Writing mpi3.py


In [7]:
!mpiexec --allow-run-as-root -n 2 python mpi3.py

Process 0 before receiving has the number 0.0
Process 1 drew the number 0.8537752809094779
Process 0 received the number 0.8537752809094779


More Send and Recv

In [8]:
%%writefile mpi4.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=0)
        print("Process", rank, "received the number", randNum[0])
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Send(randNum, dest=1)
        

Writing mpi4.py


In [9]:
!mpiexec --allow-run-as-root -n 2 python mpi4.py

Process 1 drew the number 0.9066316130726192
Process 0 before receiving has the number 0.0
Process 0 received the number 0.9066316130726192
Process 1 received the number 1.8132632261452384


#Non-blocking Communication

In [10]:
%%writefile mpi5.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
import time

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        req = comm.Isend(randNum, dest=0)
        req.Wait()
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        #time.sleep(1)
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])

Writing mpi5.py


In [11]:
!mpiexec --allow-run-as-root -n 2 python mpi5.py
"""No waits"""

Process 0 before receiving has the number 0.0
Process 1 drew the number 0.880923097145403
Process 0 received the number 0.880923097145403


'No waits'

In [12]:
!mpiexec --allow-run-as-root -n 2 python mpi5.py

Process 0 before receiving has the number 0.0
Process 1 drew the number 0.9710295363717119
Process 0 received the number 0.9710295363717119


##Non-blocking communiction

In [13]:
%%writefile mpi6.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
diffNum = numpy.random.random_sample(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Isend(randNum, dest=0)
        diffNum /= 3.14 # overlap communication
        print("diffNum=", diffNum[0])
        req = comm.Irecv(randNum, source=0)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Isend(randNum, dest=1)

Writing mpi6.py


In [14]:
!mpiexec --allow-run-as-root -n 2 python mpi6.py

Process 0 before receiving has the number 0.0
Process 1 drew the number 0.45176646507890983
diffNum= 0.2257271907829046
Process 0 received the number 0.45176646507890983
Process 1 received the number 0.9035329301578197


##Overlapping communication and computation